<a href="https://colab.research.google.com/github/AguaClara/aide_design_specs/blob/validation/validation/LFOM_Validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install aguaclara

     |████████████████████████████████| 92kB 2.3MB/s 
     |████████████████████████████████| 215kB 8.6MB/s 
  Created wheel for aguaclara: filename=aguaclara-0.2.4-cp36-none-any.whl size=99459 sha256=2d175932374ca406b97f1f4a1cbfee3f031bb0596fc902ac531880c1b698b73b
  Stored in directory: /root/.cache/pip/wheels/13/8f/96/21ca9dd731bb0b86c16855c39ce375604b9a745f3ed388ac51
Successfully built aguaclara


# Welcome to the LFOM Validation Example!
I'll try to walk through the LFOM as an example of what the Python validation tool will do. This will assume the variables have already been parsed from 
Onshape using the functions ins [parse.py](https://github.com/AguaClara/aide_design_specs/blob/master/aide_doc/form_submit/templates/docs/parse.py)

## Imports and Defining Variables
First, I'll define the variables which are output by the Onshape model, as
well as the inputs which we are trying to "calculate back to." As mentioned above, these values would be obtained through functions in `parse.py`,
but to keep the example simple I will hardcode them. Pint will be imported to
add units to the variables.

In [4]:
# import core aguaclara pkg functions
import aguaclara.core.constants as con
import aguaclara.core.physchem as pc
from aguaclara.design.lfom import LFOM
from aguaclara.core.units import u

# import numpy for arrays
import numpy as np

# define the inputs to the Onshape model
q = 10 * u.L / u.s
temp = 21 * u.degC

# define variables that will be output by Onshape
hl = 0.2 * u.m  # head loss
d_orifices = 0.016 * u.m  # diameter of orifices
b_rows = 0.017 * u.m  # space between rows
n_orifices = [17, 4, 6, 3, 4, 3, 3, 3, 3, 2, 3, 1]  # number of orifices per row
# height of the center of each row from the bottom of the bottom row
h_orifices = [0.0079375, 0.02467613636363637, 0.04141477272727274, 
              0.0581534090909091, 0.07489204545454548, 0.09163068181818185, 
              0.1083693181818182, 0.1251079545454546, 0.14184659090909096,
              0.15858522727272734, 0.1753238636363637, 0.19206250000000008] * u.m 

# set an acceptable tolerance 
# (I'll have to think harder about what this is in the future, for now I just picked 5%)
tol = 0.05

In [29]:
# redefine the inputs from this design challenge:
# https://colab.research.google.com/github/AguaClara/CEE4520/blob/master/DC%20Solutions%202017/Jupyter%20Notebooks/DC_Flow_Control_and_Measurement_Solution.ipynb#scrollTo=azQlMZSDE6jB
q = 31 * u.L / u.s
temp = 21 * u.degC

# define variables that were output in above DC
hl = 0.2 * u.m  # head loss
d_orifices = 0.015 * u.m  # diameter of orifices
b_rows = 0.2 * u.m  # space between rows
n_orifices = [ 43, 43, 14, 15, 13, 12, 11, 11, 9, 10] # number of orifices per row
# height of the center of each row from the bottom of the bottom row
h_orifices = [0.75, 2.75, 4.75, 6.75, 8.75, 10.75, 12.75, 14.75, 16.75, 18.75] * u.cm 

# set an acceptable tolerance 
# (I'll have to think harder about what this is in the future, for now I just picked 5%)
tol = 0.05

## Verify Basic Constraints
What exactly gets verified will vary widely depending on the component. For each
one, some basic checks can be made based on reversing calculations in the [aguaclara package](https://github.com/AguaClara/aguaclara)

For example, in [lfom.py](https://github.com/AguaClara/aguaclara/blob/master/aguaclara/design/lfom.py#L66) there is a function:


```
@property
def row_b(self):
    """The distance center to center between each row of orifices."""
    return self.hl / self.row_n
```

So to reverse that calculation we could check that `hl == row_n * row_b`.

However, we want to make sure to include the tolerance, so remembering that
the percent error equation is: 

`% error = ((approx - actual) / actual) * 100`,
we can do the following:

In [30]:
assert tol > ((len(n_orifices) * b_rows) - hl) / hl
assert -tol < ((len(n_orifices) * b_rows) - hl) / hl

# define an LFOM object to be used for some calculations
lfom = LFOM(q=q, hl=hl)

assert tol > (d_orifices - lfom.orifice_d).to(u.inch) / lfom.orifice_d
assert -tol < (d_orifices - lfom.orifice_d).to(u.inch) / lfom.orifice_d


AssertionError: ignored

## Calculate the Onshape Model's Inputs
These will almost always be temperature and flow rate, but may include other
parameters based on the component. Again, we are essentially going to reverse the calculations found in the [aguaclara package](https://github.com/AguaClara/aguaclara)

In the case of the LFOM, we'll reverse this equation:

```
@property
def orifice_n_per_row(self):
    """The number of orifices at each level."""
    h = self.row_b - 0.5*self.orifice_d
    flow_per_orifice = pc.flow_orifice_vert(self.orifice_d, h,
                                            con.VC_ORIFICE_RATIO)
    n = np.zeros(self.row_n)
    for i in range(self.row_n):
        flow_needed = self.q_per_row[i] - self.q_submerged(i, n)
        n_orifices_real = (flow_needed / flow_per_orifice).to(u.dimensionless)
        n[i] = min((max(0, round(n_orifices_real))),
                   self.orifice_n_max_per_row)
    return n
```



In [33]:
h = b_rows - 0.5 * d_orifices
flow_per_orifice = pc.flow_orifice_vert(d_orifices, h, con.VC_ORIFICE_RATIO)
q_row =  q_req = np.zeros(len(n_orifices)) * u.m ** 3 / u.s

for i in range(len(n_orifices)):
  q_req = n_orifices[i] * flow_per_orifice
  q_row[i] = q_req + lfom.q_submerged(i, n_orifices)

Now that we have the array `q_row`, we can see from the `aguaclara` package that
`q_row[n]` will simply be `q`. 

```
@property
def q_per_row(self):
    """An array of flow at each row."""
    return np.linspace(1 / self.row_n, 1, self.row_n)*self.q
```

Therefore:

In [34]:
# change to include tolerance
# assert q_row[len(n_orifices)-1] == q
print("The expected flow rate, {!s}, differed widely from the one calculated by this validation code, {!s}.".format(q, q_row[len(n_orifices)-1].to(u.L / u.s)))

The expected flow rate, 31 liter / second, differed widely from the one calculated by this validation code, 51.66 liter / second.
